In [45]:
import torch
import numpy as np

import os
import sys
sys.path.append(os.path.abspath(".."))

from ase import Atoms
from ase.visualize import view
from ase.units import GPa
from ase.spacegroup import crystal

from mattersim.forcefield.potential import Potential
from mattersim.forcefield.potential import MatterSimCalculator
from mattersim.applications.relax import Relaxer
from mattersim.applications.moldyn import MolecularDynamics

from utils.visualisation import plot_potential, plot_relaxation, visualise_structure

from ase.visualize import view

Molecular Dynamics simulation only accept ase atoms with an attached calculator.

*Args:*
- atoms (Union[Atoms, Structure]): ASE atoms object contains structure information and calculator.
- ensemble (str, optional): Simulation ensemble choosen. Defaults to nvt_nose_hoover'
- temperature (float, optional): Simulation temperature, in Kelvin. Defaults to 300 K.
- timestep (float, optional): The simulation time step, in fs. Defaults to 1 fs.
- taut (float, optional): Characteristic timescale of the thermostat, in fs. If is None, automatically set it to 1000 * timestep.
- trajectory (Union[str, Trajectory], optional): Attach trajectory object. If trajectory is a string a Trajectory will be constructed. Defaults to None, which means for no trajectory.
- logfile (str, optional): If logfile is a string, a file with that name will be opened. Defaults to '-', which means output to stdout.
- loginterval (int, optional): Only write a log line for every loginterval time steps. Defaults to 10.
- append_trajectory (bool, optional): If False the trajectory file to be overwriten each time the dynamics is restarted from scratch. If True, the new structures are appended to the trajectory file instead.


In [46]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"MatterSim running on {device}")

MatterSim running on cuda


In [47]:
model = 1

In [48]:
# Define the L1₀ unit cell parameters
a = 3.85  # Lattice constant in x and y
c = 3.72  # Lattice constant in z (slightly compressed)
alpha, beta, gamma = 90, 90, 90


# Define the L1₀ structure using spacegroup
structure = crystal(
    symbols=['Fe', 'Pt'],
    basis=[[0, 0, 0], [0.5, 0.5, 0.5]],  # Atomic positions as fractional coordinates
    spacegroup=123,  # P4/mmm
    cellpar=[a, a, c, alpha, beta, gamma]  # a, b, c, alpha, beta, gamma
)

In [49]:
# # Define the L1₀ unit cell parameters
# a = 3.85  # Lattice constant in x and y
# c = 3.72  # Lattice constant in z (slightly compressed)

# # Define the atomic positions and symbols
# positions = [
#     (0, 0, 0),               # Fe atom
#     (0.5 * a, 0.5 * a, 0.5 * c),  # Pt atom
# ]

# symbols = ['Fe', 'Pt']

# # Create the unit cell for L1₀ FePt
# structure = Atoms(symbols="FePt",
#                  positions=positions,
#                  cell=[(a, 0, 0), (0, a, 0), (0, 0, c)],
#                  pbc=True)  # Periodic boundary conditions

In [50]:
_ = visualise_structure(structure)

Structure positions: [[0.    0.    0.   ]
 [1.925 1.925 1.86 ]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [51]:
view(structure, viewer='x3d')

In [52]:
structure.positions

array([[0.   , 0.   , 0.   ],
       [1.925, 1.925, 1.86 ]])

In [53]:
structure.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)

In [54]:
print(f"Energy (eV)                 = {structure.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {structure.get_potential_energy()/len(structure)}")
print(f"Forces of first atom (eV/A) = {structure.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {structure.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {structure.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.895933151245117
Energy per atom (eV/atom)   = -5.447966575622559
Forces of first atom (eV/A) = [ 4.7683716e-07  8.0466270e-07 -1.0281801e-06]
Stress[0][0] (x-x) (eV/A^3) = 0.14023474151532517
Stress[0][0] (x-x) (GPa)    = 22.468082427978516


In [55]:
# initialize the relaxation object
relaxer = Relaxer(
    optimizer="BFGS", # the optimization method
    filter = "FrechetCellFilter",
    # filter="ExpCellFilter"
    # filter = None, # filter to apply to the cell
    constrain_symmetry=True, # whether to constrain the symmetry
)
pressure_in_GPa=1*GPa
relaxed_structure = relaxer.relax(structure, params_filter={"scalar_pressure": pressure_in_GPa}, steps=500)

      Step     Time          Energy          fmax
BFGS:    0 04:01:23      -10.551778        4.041101
BFGS:    1 04:01:23      -11.253890        4.089189
BFGS:    2 04:01:23      -13.592111        3.435768
BFGS:    3 04:01:23      -14.646073        0.485739
BFGS:    4 04:01:23      -14.654070        0.285197
BFGS:    5 04:01:23      -14.656443        0.095655
BFGS:    6 04:01:23      -14.656625        0.083116
BFGS:    7 04:01:23      -14.659590        0.224771
BFGS:    8 04:01:23      -14.659286        0.526468
BFGS:    9 04:01:23      -14.665280        0.293092
BFGS:   10 04:01:23      -14.668663        0.246912
BFGS:   11 04:01:23      -14.669966        0.192797
BFGS:   12 04:01:23      -14.673579        0.028816
BFGS:   13 04:01:23      -14.673763        0.008024


In [56]:
structure.get_positions()

array([[0.        , 0.        , 0.        ],
       [1.59480482, 1.59480482, 1.40066939]])

In [57]:
print(f"Energy (eV)                 = {structure.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {structure.get_potential_energy()/len(structure)}")
print(f"Forces of first atom (eV/A) = {structure.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {structure.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {structure.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -14.851644515991211
Energy per atom (eV/atom)   = -7.4258222579956055
Forces of first atom (eV/A) = [ 4.6566129e-08 -2.0489097e-08  6.4144842e-08]
Stress[0][0] (x-x) (eV/A^3) = -0.006397128551443001
Stress[0][0] (x-x) (GPa)    = -1.0249329805374146


In [58]:
_ = visualise_structure(structure)

Structure positions: [[0.         0.         0.        ]
 [1.59480482 1.59480482 1.40066939]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [59]:
structure.cell.cellpar()

array([ 3.18960964,  3.18960964,  2.80133877, 90.        , 90.        ,
       90.        ])

In [60]:
# trajectory_file = "md_trajectory.traj"
# log_file = "md_simulation.log"
md = MolecularDynamics(
    atoms=structure,
    ensemble="nvt_nose_hoover",
    # ensemble = "nvt_berendsen",
    temperature=300,
    timestep=1.0,
    # loginterval=10,
    # logfile = log_file,
    # trajectory=trajectory_file  # Save trajectory
)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]


In [61]:
cell = md.atoms.get_cell()
upper_triangular_cell = np.triu(cell)  # Force upper triangular structure
md.atoms.set_cell(upper_triangular_cell, scale_atoms=True)

In [62]:
md.run(n_steps=1000)

0.0100         -14.7741     -14.8414       0.0673   260.3
0.0200         -14.7741     -14.8159       0.0418   161.6
0.0300         -14.7741     -14.7886       0.0146    56.3
0.0400         -14.7741     -14.7745       0.0004     1.7
0.0500         -14.7741     -14.7814       0.0073    28.2
0.0600         -14.7741     -14.8053       0.0313   120.9
0.0700         -14.7741     -14.8332       0.0591   228.7
0.0800         -14.7741     -14.8500       0.0759   293.7


0.0900         -14.7741     -14.8470       0.0730   282.2
0.1000         -14.7741     -14.8259       0.0519   200.6
0.1100         -14.7741     -14.7978       0.0237    91.8
0.1200         -14.7741     -14.7778       0.0038    14.5
0.1300         -14.7741     -14.7770       0.0029    11.3
0.1400         -14.7741     -14.7958       0.0217    83.9
0.1500         -14.7741     -14.8237       0.0496   191.7
0.1600         -14.7741     -14.8455       0.0714   276.1
0.1700         -14.7741     -14.8497       0.0756   292.6
0.1800         -14.7741     -14.8344       0.0603   233.2
0.1900         -14.7741     -14.8076       0.0335   129.6
0.2000         -14.7741     -14.7837       0.0096    37.2
0.2100         -14.7741     -14.7758       0.0017     6.6
0.2200         -14.7741     -14.7882       0.0141    54.7
0.2300         -14.7741     -14.8140       0.0399   154.5
0.2400         -14.7741     -14.8389       0.0648   250.8
0.2500         -14.7741     -14.8496       0.0755   292.1
0.2600        

In [63]:
md.atoms.cell.cellpar()

array([ 3.18960964,  3.18960964,  2.80133877, 90.        , 90.        ,
       90.        ])

In [64]:
md.atoms.get_positions()

array([[-0.00214921, -0.00261349, -0.00682642],
       [ 1.59542001,  1.595553  ,  1.40262352]])

In [65]:
print(f"Energy (eV)                 = {md.atoms.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {md.atoms.get_potential_energy()/len(structure)}")
print(f"Forces of first atom (eV/A) = {md.atoms.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {md.atoms.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {md.atoms.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -14.851388931274414
Energy per atom (eV/atom)   = -7.425694465637207
Forces of first atom (eV/A) = [0.01742537 0.02118938 0.04473238]
Stress[0][0] (x-x) (eV/A^3) = -0.006444543618466759
Stress[0][0] (x-x) (GPa)    = -1.0325297117233276


In [66]:
_ = visualise_structure(md.atoms)

Structure positions: [[-0.00214921 -0.00261349 -0.00682642]
 [ 1.59542001  1.595553    1.40262352]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.